In [4]:
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as T
import torchvision.models as models
import cs231n.datasets.local_CIFAR10 as dst
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import sampler

In [5]:
def conv3x3(in_planes, out_planes, stride=1,padding=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=padding, bias=False)

In [6]:
def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

In [8]:
transform  = T.Compose([T.ToTensor(),T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
device = torch.device('cuda')
train_data = dst.local_CIFAR10("C:/Users/wtser/Desktop/learnData/data/",download=False,train=True,
                               transform=transform)
loader_train = DataLoader(train_data,batch_size=128,shuffle=False)
test_data = dst.local_CIFAR10("C:/Users/wtser/Desktop/learnData/data/",download=False,train=False,
                              transform=transform)
loader_test = DataLoader(test_data,batch_size=128,shuffle=False)

In [13]:
for x,y in loader_train:
    print(torch.min(x))
    break

tensor(-2.4291)


In [5]:
class ResBlock(torch.nn.Module):
    "残差块"
    def __init__(self,indim,outdim,downSampler=None,stride=1,norm_layer = None):
        super(ResBlock,self).__init__()
        if norm_layer == None:
            self.norm_layer = nn.BatchNorm2d
        
        self.conv1 = conv3x3(indim,outdim,stride)
        self.bn1 = self.norm_layer(outdim)
        self.relu = nn.ReLU(inplace=True)
        self.con2 = conv3x3(outdim,outdim)
        self.bn2 = self.norm_layer(outdim)
        self.downSampler = downSampler
        self.stride = stride
    
    def forward(self,x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.con2(out)
        out = self.bn2(out)
        
        if self.downSampler != None:
            identity = self.downSampler(x)
        
        out += identity
        out = self.relu(out)
        
        return out

In [6]:
class Resnet(torch.nn.Module):
    
    def __init__(self,block,layers,num_classes,padding = None,norm_layer = None):
        super(Resnet,self).__init__()
        if norm_layer == None:
            self.norm_layer = nn.BatchNorm2d
        self.indim = 3
        outdim = 64
        self.conv1 = nn.Conv2d(in_channels=self.indim,out_channels=outdim,bias=False,kernel_size=7,padding=3
                               ,stride=2)
        self.indim = outdim
        self.bn1 = self.norm_layer(outdim)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(3,stride=2,padding=1)
        self.layer1 = self.make_layers(block,64,layers[0])
        self.layer2 = self.make_layers(block,128,layers[1],stride=2)
        self.layer3 = self.make_layers(block,256,layers[2],stride=2)
        self.layer4 = self.make_layers(block,512,layers[3],stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(in_features=512,out_features=num_classes)

    def forward(self,x):
        out = self.conv1(x)   
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = out.view(out.shape[0],-1)
        out = self.fc(out)
        return out
    
    def make_layers(self,block,outdim,block_num,stride=1):
        downsample = None
        if stride != 1 or self.indim != outdim:
            downsample = nn.Sequential(conv1x1(self.indim,outdim,stride=stride),self.norm_layer(outdim))
        
        layers = []
        layers.append(block(self.indim,outdim,downSampler=downsample,stride=stride))
        self.indim = outdim
        
        for _ in range(1,block_num):
            layers.append(block(self.indim,outdim))
        
        return nn.Sequential(*layers)
        
        

In [29]:
resnet = Resnet(block=ResBlock,layers=[3,4,6,3],num_classes=10)
resnet = resnet.cuda()

In [52]:
num_epoch = 10
learn_rate = 1e-3
momentum = 0.9
total_step = len(loader_train)
optimer = optim.Adam(resnet.parameters(),lr=learn_rate)
criterion = nn.CrossEntropyLoss()

In [50]:
for epoch in range(num_epoch):
    for index,(x,y) in enumerate(loader_train):
        x = x.to(device)
        y = y.to(device)
        resnet.train()
        out = resnet(x)
        loss = criterion(out,y)
        
        optimer.zero_grad()
        loss.backward()
        optimer.step()
        if index % 100 == 0:
            print('Epoch[{}/{}],Step[{}/{}],loss{:.4f}'.format(epoch+1, num_epoch, index+1, total_step, loss.item()))
            check_acc(resnet,loader_test)
check_acc(resnet,loader_test)

Epoch[1/10],Step[1/391],loss0.1291
Test accuracy of model on 10000 of test images:76.68%
Epoch[1/10],Step[101/391],loss0.4079
Test accuracy of model on 10000 of test images:76.86%
Epoch[1/10],Step[201/391],loss0.3526
Test accuracy of model on 10000 of test images:76.77%
Epoch[1/10],Step[301/391],loss0.1570
Test accuracy of model on 10000 of test images:76.7%
Test accuracy of model on 10000 of test images:76.74%


In [10]:
def check_acc(model,test_data):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
    for i,(x,y) in enumerate(test_data):
        x = x.to(device)
        y = y.to(device)
        out = model(x)
        _,pre = torch.max(out,1)
        total += y.shape[0]
        correct += (pre==y).sum().item()
    print('Test accuracy of model on 10000 of test images:{}%'.format(100*correct/total ))